### Web Scrapping - **Mayo Clinic**

- `https://www.mayoclinic.org/es/diseases-conditions`

In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import re
import os

In [34]:
url = 'https://www.mayoclinic.org/es/diseases-conditions'
driver_path = os.path.join(os.getcwd(), 'drivers','chromedriver.exe' )
print(driver_path)
# creamos el servicio
service = Service(executable_path=driver_path)
driver = webdriver.Chrome(service=service)
driver.get(url)
time.sleep(2)


e:\NLP_proyectos\proyecto_Final\entrega02\web_scrapping\drivers\chromedriver.exe


In [ ]:
# Creamos una lista con palabras de busqueda de enfermedades
enfermedades = []
sintomas = []


def scroll_to_element(driver, xpath):
    try:
        element = driver.find_element(By.XPATH, xpath)
        driver.execute_script("arguments[0].scrollIntoView();", element)
        time.sleep(1)
    except Exception as e:
        print(f"Error al hacer scroll: {e}")

def get_sintomas_text(driver):
    try:
        # Buscar la sección que tiene el aria-labelledby="síntomas"
        sintomas_section = driver.find_element(By.XPATH, "//section[@aria-labelledby='síntomas']")
        # Scroleamos hasta la sección
        scroll_to_element(driver, "//section[@aria-labelledby='síntomas']")
        print(sintomas_section.text)
        return sintomas_section.text.strip() if sintomas_section.text else "No se encontraron síntomas"

    except Exception as e:
        print(f"Tipo de pagina 2")
        try:
            # Verificar si la sección de síntomas está presente con un XPath más robusto
            sintomas_h2 = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//h2[contains(text(), 'Síntomas')]"))
            )
            print(sintomas_h2.text)
            # Obtener el siguiente contenido que no sea otro h2 (es decir, texto)
            texto_entre_h2_h3 = driver.find_elements(By.XPATH, 
                "//h2[contains(text(), 'Síntomas')]/following-sibling::*[not(self::h2)]"
            )
         # Scroleamos hasta la sección
            scroll_to_element(driver, "//h2[contains(text(), 'Síntomas')]")

            texto = ""
            for elemento in texto_entre_h2_h3:
                texto += elemento.text + "\n"
            
            # Si no hay texto, devuelve el mensaje adecuado
            return texto.strip().split('\n\n')[0] if texto else "No se encontraron síntomas"
        except Exception as e:
            print(f"Error al obtener síntomas:")
# /html/body/div[1]/d

# /html/body/div[1]/div/div/div[3]/main/div/div[1]/div/div[1]/div[2]/div/div[2]/div/div[2]/div[3]/div[2]/div/div/ul/li[1]/div/a/span
# /html/body/div[1]/div/div/div[3]/main/div/div[1]/div/div[1]/div[2]/div/div[2]/div/div[2]/div[3]/div[2]/div/div/ul/li[2]/div/a/span

# /html/body/div[1]/div/div/div[3]/main/div/div[1]/div/div[1]/div[2]/div/div[2]/div/div[2]/div[3]/div[2]/div/div/ul/li[22]/div/a/span

# Recorremos la pagina para obtener las enfermedades
for i in range(1, 23):
    try:
        letra_button = driver.find_element(By.XPATH, f'/html/body/div[1]/div/div/div[3]/main/div/div[1]/div/div[1]/div[2]/div/div[2]/div/div[2]/div[3]/div[2]/div/div/ul/li[{i}]/div/a/span')
        # Hacemos click en el boton
        letra_button.click()
        time.sleep(1)
    except:
        print('Error al obtener las enfermedades')
        pass
    # Obtenemos las enfermedades
    try:
        secciones = driver.find_elements(By.XPATH, '/html/body/div[1]/div/div/div[3]/main/div/div[1]/div/div[2]/div[2]/div[1]/div[2]/div/div')
    except:
        print('Error al obtener las secciones')
        
    try:
        uls = secciones[0].find_elements(By.TAG_NAME, 'ul')
        shape_ul = len(uls)
        print(f'shape_ul:{shape_ul}')
    except:
        print('Error al obtener las uls')

    k = 300
    for i in range(1, shape_ul+1):#shape_ul+1):
        # Calculamos la cantidad de enfermedades (li) que hay en cada ul
        try:
            time.sleep(1)
            shape_li = len(uls[i-1].find_elements(By.TAG_NAME, 'li'))
        # Cada que cambiamos de ul, scroleamos hacia abajo
        except Exception as e:
            #print(uls[i-1])
            print(f'Error al obtener las lis')
            shape_li = 20

        k += 300
        print('seccion: {}'.format(i))

        time.sleep(1)
        print(f'shape_li:{shape_li}')
        for j in range(1, shape_li+1):  
            print(f'li: {j}')  
            driver.execute_script(f"window.scrollTo(0, {k})")  
            time.sleep(2)          
            try: 
                enf = driver.find_element(By.XPATH, f'/html/body/div[1]/div/div/div[3]/main/div/div[1]/div/div[2]/div[2]/div[1]/div[2]/div/div/ul[{i}]/li[{j}]/div/div/a')
                enfermedades.append(enf.text)
                # Hacemos click en la enfermedad
                print(enf.text)
            except:
                print('Error al obtener la enfermedad')
                break
                
        
            try:
                enf.click()
            except:
                print('Error al hacer click en la enfermedad')
                pass
            

            try:
                sintoma = get_sintomas_text(driver)
                sintomas.append(sintoma)
                # Regresamos a la pagina anterior
                print('Sintoma: {}'.format(sintoma[:10]))
                if sintoma:
                    driver.back()
                
            except:
                print('Error al obtener los sintomas')
                driver.back()
                    

# Convertimos a dataframe
df = pd.DataFrame({'enfermedad': enfermedades, 'sintomas': sintomas})
# Guardamos el dataframe en un archivo csv
df.to_csv('enfermedades_2.csv', index=False)





#div/div/div[3]/main/div/div[1]/div/div[2]/div[2]/div[1]/div[2]/div/div/ul[1]/li[1]/div/div/a

# /html/body/div[1]/div/div/div[3]/main/div/div[1]/div/div[2]/div[2]/div[1]/div[2]/div/div/ul[1]/li[2]/div/div/a

# 

# /html/body/div[1]/div/div/div[3]/main/div/div[1]/div/div[2]/div[2]/div[1]/div[2]/div/div/ul[2]/li[1]/div/div/a
# /html/body/div[1]/div/div/div[3]/main/div/div[1]/div/div[2]/div[2]/div[1]/div[2]/div/div/ul[2]/li[2]/div/div/a





# /html/body/div[1]/div/div/div[3]/main/div/div[1]/div/div[2]/div[2]/div[1]/div[2]/div/div/ul[13]/li/div/div/a
# /html/body/div[1]/div/div/div[3]/main/div/div[1]/div/div[2]/div[2]/div[1]/div[2]/div/div/ul[2]/li[1]/div/div

shape_ul:13
seccion: 1
shape_li:3
li: 1
Aborto espontáneo
Tipo de pagina 2
Síntomas
Sintoma: La mayoría
li: 2
Absceso dental
Síntomas
Los signos y los síntomas de un absceso dental incluyen los siguientes:
Dolor de muela intenso, constante y palpitante que puede extenderse hasta la mandíbula, el cuello o el oído
Dolor o molestia ante temperaturas frías y calientes
Dolor o molestia al masticar o morder
Fiebre
Hinchazón en la cara, las mejillas o el cuello que puede provocar dificultad para respirar o tragar
Inflamación y sensibilidad en los ganglios linfáticos debajo de la mandíbula o en el cuello
Olor desagradable en la boca
La boca se inunda repentinamente de líquido salado con olor y gusto desagradables y alivio del dolor, si se rompe el absceso
Sintoma: Síntomas
L
li: 3
Abuso de drogas recetadas
Tipo de pagina 2
Síntomas
Sintoma: Los signos
seccion: 2
shape_li:8
li: 1
Acalasia
Tipo de pagina 2
Síntomas
Sintoma: Por lo gen
li: 2
Acantosis pigmentaria
Síntomas
El principal signo de la

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x00007FF678986CF5+28821]
	(No symbol) [0x00007FF6788F3880]
	(No symbol) [0x00007FF67879578A]
	(No symbol) [0x00007FF67876F4F5]
	(No symbol) [0x00007FF678816247]
	(No symbol) [0x00007FF67882ECE2]
	(No symbol) [0x00007FF67880F0A3]
	(No symbol) [0x00007FF6787DA778]
	(No symbol) [0x00007FF6787DB8E1]
	GetHandleVerifier [0x00007FF678CBFCED+3408013]
	GetHandleVerifier [0x00007FF678CD745F+3504127]
	GetHandleVerifier [0x00007FF678CCB63D+3455453]
	GetHandleVerifier [0x00007FF678A4BDFB+835995]
	(No symbol) [0x00007FF6788FEB9F]
	(No symbol) [0x00007FF6788FA854]
	(No symbol) [0x00007FF6788FA9ED]
	(No symbol) [0x00007FF6788EA1D9]
	BaseThreadInitThunk [0x00007FFF229C7344+20]
	RtlUserThreadStart [0x00007FFF22B026B1+33]


In [84]:
df['sintomas'][0]

'La mayoría de los abortos espontáneos ocurren durante el primer trimestre del embarazo, que son aproximadamente las primeras 13 semanas.\nEntre los síntomas que pueden aparecer se encuentran los siguientes:\nSangrado vaginal con o sin dolor, como un sangrado leve que se conoce como manchado.\nDolor o calambres en la zona pélvica o en la parte baja de la espalda.\nLíquido o tejido que sale de la vagina.\nLatidos rápidos del corazón.\nSi expulsaste tejido de la vagina, colócalo en un recipiente limpio. Después, llévalo al consultorio de tu profesional de atención médica o al hospital. Un laboratorio puede examinar el tejido para verificar si hay signos de aborto espontáneo.\nTen en cuenta que la mayoría de las personas embarazadas que presentan manchado o sangrado vaginal durante el primer trimestre tienen embarazos sin complicaciones. Sin embargo, llama a tu equipo de atención médica de inmediato si el sangrado es intenso o si viene acompañado de calambres.\nSolicite una consulta\n\nCa

In [85]:
df

,enfermedad,sintomas
0,Aborto espontáneo,La mayoría de los abortos espontáneos ocurren ...
1,Absceso dental,
2,Abuso de drogas recetadas,Los signos y síntomas del abuso de medicamento...


In [ ]:
# /html/body/div[1]/div/div/div[4]/div[2]/div/div/div[1]/article/div/div[2]/div/div/div/div[4]/section/div/div[1]/div[2]/div/h2

In [126]:
def extract_sintomas_text(driver):
    # Scroleamos hasta el elemento h2 que contiene "Síntomas"
    try:
        sintomas_header = driver.find_element(By.XPATH, "//h2[contains(text(), 'Síntomas')]")
        driver.execute_script("arguments[0].scrollIntoView();", sintomas_header)
        time.sleep(1)
    except:
        print("No se encontró el encabezado 'Síntomas'")
        return "No se encontró el encabezado 'Síntomas'"

    # Obtener todo el texto de la página
    page_text = driver.find_element(By.TAG_NAME, "body").text
    
    # Usar expresiones regulares para encontrar el texto entre los encabezados
    pattern = re.compile(r'Síntomas(.*?)Causas', re.DOTALL)
    match = pattern.search(page_text)
    
    if match:
        return match.group(1).strip()
    else:
        return "No se encontró el texto entre los encabezados especificados."



In [127]:
sintomas_text = extract_text_between_headers(driver, 'Síntomas', 'Causas')

In [128]:
print(sintomas_text)

y
causas
Diagnóstico y
tratamiento
Médicos y
departamentos
En esta página
Descripción general
Síntomas
Cuándo consultar al médico


In [32]:
# /html/body/form/div[6]/article/div[1]/div[1]/div[2]/h2[2] (sintomas)

sintomas = driver.find_element(By.XPATH, '/html/body/form/div[6]/article/div[1]/div[1]/div[2]/h2[2]').text  
print(sintomas)
# /html/body/form/div[6]/article/div[1]/div[1]/div[2]/h2[2]
#/html/body/div[1]/div/div/div[4]/div[2]/div/div/div[1]/article/div/div[2]/div/div/div/div[4]/section/div/div[1]/div[2]/div/h2
# /html/body/form/div[6]/article/div[1]/div[1]/div[2]/h2[2]
# /html/body/form/div[6]/article/div[1]/div[1]/div[2]/h2[2]
# Obtener el texto entre el h2 y h3
texto_entre_h2_h3 = driver.find_elements(By.XPATH, 
    '/html/body/form/div[6]/article/div[1]/div[1]/div[2]/h2[2]/following-sibling::*[not(self::h2[3])]')

# Extraer el texto de los elementos encontrados
for elemento in texto_entre_h2_h3:
    print('Elemento:  ',elemento.text)
#/html/body/form/div[6]/article/div[1]/div[1]/div[2]/h3

# /html/body/form/div[6]/article/div[1]/div[1]/div[2]/h2[3]
# Creamos un funcion para obtener solo los síntomas
def get_sintomas(driver):
    # Buscar los elementos entre el h2[2] y el siguiente h2[3] (sin incluirlos)
    sintomas = driver.find_elements(By.XPATH, 
        '/html/body/form/div[6]/article/div[1]/div[1]/div[2]/h2[2]/following-sibling::*[not(self::h2[3])]')

    # Inicializamos una variable para acumular el texto
    texto = ''
    
    # Recorrer todos los elementos encontrados y concatenar su texto
    for elemento in sintomas:
        texto += elemento.text + '\n'  # Usamos += para concatenar texto y agregamos un salto de línea
    
    return texto


Síntomas
Elemento:   La mayoría de los abortos espontáneos ocurren durante el primer trimestre del embarazo, que son aproximadamente las primeras 13 semanas.
Elemento:   Entre los síntomas que pueden aparecer se encuentran los siguientes:
Elemento:   Sangrado vaginal con o sin dolor, como un sangrado leve que se conoce como manchado.
Dolor o calambres en la zona pélvica o en la parte baja de la espalda.
Líquido o tejido que sale de la vagina.
Latidos rápidos del corazón.
Elemento:   Si expulsaste tejido de la vagina, colócalo en un recipiente limpio. Después, llévalo al consultorio de tu profesional de atención médica o al hospital. Un laboratorio puede examinar el tejido para verificar si hay signos de aborto espontáneo.
Elemento:   Ten en cuenta que la mayoría de las personas embarazadas que presentan manchado o sangrado vaginal durante el primer trimestre tienen embarazos sin complicaciones. Sin embargo, llama a tu equipo de atención médica de inmediato si el sangrado es intenso o s

In [67]:
url = 'https://www.mayoclinic.org/es/diseases-conditions'
driver_path = os.path.join(os.getcwd(), 'drivers','chromedriver.exe' )
print(driver_path)
# creamos el servicio
service = Service(executable_path=driver_path)
driver = webdriver.Chrome(service=service)
driver.get(url)
time.sleep(2)

e:\NLP_proyectos\proyecto_Final\entrega02\web_scrapping\drivers\chromedriver.exe


In [ ]:
try:
    sintomas = driver.find_element(By.XPATH, '/html/body/form/div[6]/article/div[1]/div[1]/div[2]/h2[2]').text  
    texto_entre_h2_h3 = driver.find_elements(By.XPATH, 
    '/html/body/form/div[6]/article/div[1]/div[1]/div[2]/h2[2]/following-sibling::*[not(self::h2[3])]')
except:
    sintomas = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[4]/div[2]/div/div/div[1]/article/div/div[2]/div/div/div/div[4]/section/div/div[1]/div[2]/div/h2').text
    ul_sintomas = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[4]/div[2]/div/div/div[1]/article/div/div[2]/div/div/div/div[4]/section/div/div[2]/div/ul')
print(sintomas)
# /html/body/form/div[6]/article/div[1]/div[1]/div[2]/h2[2]
#/html/body/div[1]/div/div/div[4]/div[2]/div/div/div[1]/article/div/div[2]/div/div/div/div[4]/section/div/div[1]/div[2]/div/h2
# /html/body/form/div[6]/article/div[1]/div[1]/div[2]/h2[2]
# /html/body/form/div[6]/article/div[1]/div[1]/div[2]/h2[2]
# Obtener el texto entre el h2 y h3


# Extraer el texto de los elementos encontrados
for elemento in texto_entre_h2_h3:
    print('Elemento:  ',elemento.text)

Síntomas


In [93]:
# Creamos un funcion para obtener solo los síntomas
def get_sintomas(driver):
    try:
        # Scroleamos hasta el elemento h2[2]
        driver.execute_script("arguments[0].scrollIntoView();", driver.find_element(By.XPATH, '/html/body/form/div[6]/article/div[1]/div[1]/div[2]/h2[2]'))
    except:
        driver.execute_script("arguments[0].scrollIntoView();", driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[4]/div[2]/div/div/div[1]/article/div/div[2]/div/div/div/div[4]/section/div/div[1]/div[2]/div/h2'))
        print('Error al scrollear')
    time.sleep(1)
    # Buscar los elementos entre el h2[2] y el siguiente h2[3] (sin incluirlos)

    try:
        sintomas = driver.find_element(By.XPATH, '/html/body/form/div[6]/article/div[1]/div[1]/div[2]/h2[2]').text  
        texto_entre_h2_h3 = driver.find_elements(By.XPATH, 
        '/html/body/form/div[6]/article/div[1]/div[1]/div[2]/h2[2]/following-sibling::*[not(self::h2[3])]')

            # Inicializamos una variable para acumular el texto
        texto = ''
        
        # Recorrer todos los elementos encontrados y concatenar su texto
        for elemento in texto_entre_h2_h3:
            texto += elemento.text + '\n'  # Usamos += para concatenar texto y agregamos un salto de línea

        # Filtramos el texto para obtener el texto antes 'Causas' 
        try:
            texto = texto.split('Causas')[0]
        except:
            print('No se encontró la palabra "Causas"')
            pass

        if texto == '':
            texto = 'No se encontraron síntomas'
        return texto
    except:
        sintomas = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[4]/div[2]/div/div/div[1]/article/div/div[2]/div/div/div/div[4]/section/div/div[1]/div[2]/div/h2').text
        ul_sintomas = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[4]/div[2]/div/div/div[1]/article/div/div[2]/div/div/div/div[4]/section/div/div[2]/div/ul').text
        print(ul_sintomas[:10])

        ul_sintomas = ul_sintomas.split('Causas')[0]

        return ul_sintomas
    



In [94]:
print(get_sintomas(driver))

La mayoría de los abortos espontáneos ocurren durante el primer trimestre del embarazo, que son aproximadamente las primeras 13 semanas.
Entre los síntomas que pueden aparecer se encuentran los siguientes:
Sangrado vaginal con o sin dolor, como un sangrado leve que se conoce como manchado.
Dolor o calambres en la zona pélvica o en la parte baja de la espalda.
Líquido o tejido que sale de la vagina.
Latidos rápidos del corazón.
Si expulsaste tejido de la vagina, colócalo en un recipiente limpio. Después, llévalo al consultorio de tu profesional de atención médica o al hospital. Un laboratorio puede examinar el tejido para verificar si hay signos de aborto espontáneo.
Ten en cuenta que la mayoría de las personas embarazadas que presentan manchado o sangrado vaginal durante el primer trimestre tienen embarazos sin complicaciones. Sin embargo, llama a tu equipo de atención médica de inmediato si el sangrado es intenso o si viene acompañado de calambres.
Solicite una consulta


